In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import sys
import tsplib95

In [ ]:
problem = tsplib95.load('./symmetric_problems/gr21.tsp')
# problem = tsplib95.load('./symmetric_problems/berlin52.tsp')
G = problem.get_graph()

In [ ]:
def getMin(G, mstFlag):
    min_edge = 0
    min = sys.maxsize  # assigning largest numeric value to min
    for i in [(u, v, edata['weight']) for u, v, edata in G.edges( data = True) if 'length' in edata ]:
        if mstFlag[i] == False and i[2] < min:
            min = i[2]
            min_edge = i
    return min_edge

In [ ]:
def findRoot(parent, i):
    if parent[i] == i:
        return i
    return findRoot(parent, parent[i])

In [ ]:
def union(parent, order, x, y):
    xRoot = findRoot(parent, x)
    yRoot = findRoot(parent, y)
    # Attach smaller order tree under root of high order tree
    if order[xRoot] < order[yRoot]:
        parent[xRoot] = yRoot
    elif order[xRoot] > order[yRoot]:
        parent[yRoot] = xRoot
    # If orders are same, then make any one as root and increment its order by one
    else :
        parent[yRoot] = xRoot
        order[xRoot] += 1

In [ ]:
def minimumWeightedMatching(MST, G, odd_vert):
    while odd_vert:
        v = odd_vert.pop()
        length = float("inf")
        u = 1
        closest = 0
        for u in odd_vert:
            if G[v][u]['weight'] < length :
                length = G[v][u]['weight']
                closest = u
        MST.add_edge(v, closest, length = length)
        odd_vert.remove(closest)
  

In [ ]:
MST = nx.minimum_spanning_tree(G)

In [ ]:
MST.nodes()

In [ ]:
def christofedes(G ,pos):
    opGraph=nx.DiGraph()
    #optimal_dist = 0
    MST = nx.minimum_spanning_tree(G) # generates minimum spanning tree of graph G, using Prim's algo
    odd_vert = [] #list containing vertices with odd degree
    for i in MST.nodes():
        if MST.degree(i)%2 != 0: 
            odd_vert.append(i) #if the degree of the vertex is odd, then append it to odd_vert list
    minimumWeightedMatching(MST, G, odd_vert) #adds minimum weight matching edges to MST
    # now MST has the Eulerian circuit
    start = MST.nodes()[1]
    visited = [False] * len(MST.nodes())
    # finds the hamiltonian circuit
    curr = 1
    visited[curr] = True
    for nd in MST.neighbors(curr):
            if visited[nd] == False or nd == start:
                next = nd
                break
    while next != start:
        visited[next]=True
        opGraph.add_edge(curr,next,length = G[curr][next]['weight'])
        nx.draw_networkx_edges(G, pos, arrows = True, edgelist = [(curr, next)], width = 2.5, alpha = 0.6, edge_color = 'r')
        # optimal_dist = optimal_dist + G[curr][next]['length']
        # finding the shortest Eulerian path from MST
        curr = next
        for nd in MST.neighbors(curr):
            if visited[nd] == False:
                next = nd
                break
        if next == curr:
            for nd in G.neighbors(curr):
            if visited[nd] == False:
                    next = nd
                break
    if next == curr:
            next = start
    opGraph.add_edge(curr,next,length = G[curr][next]['weight'])
    nx.draw_networkx_edges(G, pos, edgelist = [(curr, next)], width = 2.5, alpha = 0.6, edge_color = 'r')
    # optimal_dist = optimal_dist + G[curr][next]['length']
    # print optimal_dist
    return opGraph

In [ ]:
def DrawGraph(G,color):
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels = True, edge_color = color)  #with_labels=true is to show the node number in the output graph
    edge_labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels = edge_labels,  font_size = 11) #prints weight on all the edges
    return pos

In [ ]:
pos = DrawGraph(G,'red')

In [ ]:
opGraph = christofedes(G, pos)